# Abstract

Make the test comparison associations

# Environment

In [1]:
# Setup debugging log
import logging
logger = logging.getLogger('jwst.associations')
handler = logging.StreamHandler()
logger.addHandler(handler)
handler.setLevel(logging.DEBUG)
logger.setLevel(logging.DEBUG)

In [2]:
from functools import partial
from glob import glob
from multiprocessing import Pool
from os import (
    listdir, 
    mkdir,
    path
)
import pdb
import re
import shutil
from tempfile import TemporaryDirectory

In [3]:
import numpy as np

In [4]:
from jwst.associations import (
    AssociationRegistry,
    generate
)
from jwst.associations.main import Main

In [5]:
from jwst.associations.tests.helpers import (
    combine_pools,
    registry_level3_only,
    registry_level2_only,
    t_path,
)

In [6]:
from jwst.associations.tests.test_standards import (
    DEF_ARGS,
    LV2_ONLY_ARGS,
    LV3_ONLY_ARGS,
    MakePars,
    standards
)

In [7]:
# Folders
DATA_DIR = path.join(
    '..', 'data'
)
NEW_ASN_DIR = path.join(
    DATA_DIR,
    'new_asn_standards'
)
SDP_DIR = path.join(
    '..', 'data', 'sdp'
)
SDP_POOLS = path.join(
    SDP_DIR, 'pools'
)
SDP_NEW_ASN_DIR = path.join(
    SDP_DIR, 'new_asns'
)

# Library

In [8]:
def make_standards(pool_root, 
                   main_args=None,
                   source=DATA_DIR,
                   outdir=NEW_ASN_DIR
                  ):
    """Make the association standards for a pool"""
    pool_path = path.join(
        source,
        pool_root + '.csv'
    )
    pool = combine_pools(pool_path)
    if main_args is None:
        main_args = []
    with TemporaryDirectory() as tmp_path:
        args = main_args + ['-v', '-p', tmp_path]
        results = Main(
            args,
            pool=pool
        )
        asn_paths = listdir(tmp_path)
        for asn_path in asn_paths:
            matches = re.match('[^-]+(-.+)', asn_path)
            asn_tail = matches.group(1)
            from_path = path.join(tmp_path, asn_path)
            to_path = path.join(outdir, pool_root + asn_tail)
            shutil.move(from_path, to_path)
        pool.write(path.join(outdir, pool_root + '_std.csv'))

# Main

## Clear result folder

In [9]:
try:
    shutil.rmtree(NEW_ASN_DIR)
except FileNotFoundError:
    pass

mkdir(NEW_ASN_DIR)

In [10]:
try:
    shutil.rmtree(SDP_NEW_ASN_DIR)
except FileNotFoundError:
    pass

mkdir(SDP_NEW_ASN_DIR)

## Make the standards

In [11]:
make_standards_args = [
    (standard.pool_root, standard.main_args)
    for standard in standards
]

In [12]:
def map_func(t):
    make_standards(t[0], t[1], source=DATA_DIR, outdir=NEW_ASN_DIR)

In [13]:
with Pool(6) as mp:
    mp.map(map_func, make_standards_args)

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmp41_0tplp']
Reading rules.
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpkxq1q2i5']
Reading rules.
Generating associations.
Command-line arguments: ['-r', '/Users/eisenham/Documents/ssbdev/jwst/jwst/associations/tests/../lib/rules_level3.py', '--ignore-default', '-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmplfox4oz6']
Generating associations.
Reading rules.
Generating associations.
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpmy20gt_e']
Reading rules.
Generating associations.
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpx6h4n935']
Reading rules.
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpxb18s7dj']
Reading rules.
Generating associations.
Generating associations.


201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_lev

There where 1 associations and 0 orphaned items found.
Associations found are:
jw99009-o001_image3_001_asn with 1 products
Rule=candidate_Asn_Image
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '99009'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'miri'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f560w'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['pupil', 'grating'], 'value': None})
DMSAttrConstraint({'name': 'subarray', 'sources': ['subarray'], 'value': 'full'})
Constraint_Target({'name': 'target', 'sources': ['targetid'], 'value': '1'})
Constraint_Image({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'mir_image'})
DMSAttrConstraint({'name': 'wfsvisit', 'sources': ['visitype'], 'value': 'prime_targeted_fixed'})
Constraint_Obsnum({'name': 'obs_num', 'sources': ['obs_num'], 'value': None})
DMSAttrConstraint({'name': 'acq_exp', 'sources': ['exp_type'], 'value': 'mir_tacq|nis_taconfirm|



Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpywk85b0l']
Reading rules.
Generating associations.


201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.


...

There where 48 associations and 25 orphaned items found.
Associations found are:
Association jw99009-o001_image2_001_asn
    Parameters:        Product type: indetermined        Rule:         indetermined        Program:      indetermined        Target:       indetermined        Pool:         indetermined
        DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '99009'})
DMSAttrConstraint({'name': 'is_tso', 'sources': ['tsovisit'], 'value': None})
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '99009'})
DMSAttrConstraint({'name': 'target', 'sources': ['targetid'], 'value': '1'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'nirspec'})
DMSAttrConstraint({'name': 'detector', 'sources': ['detector'], 'value': 'nrs1'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f100lp'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['grating'], 'value': 'g140m'})
DMSAttrConstraint({'name': 'subarr

201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2b.
201805292141:WARNING:jwst.associations.lib.rules_leve

201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_level2_base.rename_to_level2a:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2a.
201805292141:WARNING:jwst.associations.lib.rules_lev

201805292141:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2b.
201805292141:WARNING:jwst.associations.lib.rules_level3_base.rename_to_level2:Item FILENAME="set acid" is not a Level 1b name. Cannot transform to Level 2b.


There where 40 associations and 2 orphaned items found.
Associations found are:
jw10005-a3001_coron3_001_asn with 1 products
Rule=discover_Asn_Coron
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '10005'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'nircam'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f210m'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['pupil'], 'value': 'maskrnd'})
DMSAttrConstraint({'name': 'subarray', 'sources': ['subarray'], 'value': 'maska210r'})
DMSAttrConstraint({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'nrc_coron'})
DMSAttrConstraint({'name': 'target', 'sources': ['targetid'], 'value': '1'})
Constraint_Obsnum({'name': 'obs_num', 'sources': ['obs_num'], 'value': None})
DMSAttrConstraint({'name': 'acq_exp', 'sources': ['exp_type'], 'value': 'mir_tacq|nis_taconfirm|nis_tacq|nrc_taconfirm|nrc_tacq|nrs_confirm|nrs_taconfirm|nrs_tacq|nrs_taslit'})
DMSAttrConstra

## Make the SPD-based associations

In [14]:
pools = glob(path.join(SDP_POOLS, '*.csv'))

In [15]:
make_standard_args = [
    (path.splitext(path.split(pool)[1])[0], path.split(pool)[0])
    for pool in pools
]

In [16]:
def map_func(t):
    make_standards(t[0], source=t[1], outdir=SDP_NEW_ASN_DIR)

In [17]:
with Pool(6) as mp:
    mp.map(map_func, make_standard_args)

Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpf9jdun78']
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpqo8qd0zk']
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpf1zv0kvp']
Reading rules.
Reading rules.
Reading rules.
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpaxaewvl8']
Reading rules.
Generating associations.
Generating associations.
Generating associations.
Generating associations.
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpf4s4pgqt']
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmp053zt1au']
Reading rules.
Reading rules.
Generating associations.
Generating associations.
There where 4 associations and 0 orphaned items found.
Associations found are:
jw10004-o001_tso3_001_asn with 1 products
Rule=candidate_Asn_TSO
DMSAttrCon


Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpvborneth']
Reading rules.
Generating associations.
There where 0 associations and 12 orphaned items found.
Associations found are:
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmp8efil7kz']
Reading rules.
Generating associations.
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpbjtdr8ta']
Reading rules.
Generating associations.
There where 2 associations and 0 orphaned items found.
Associations found are:
jw90002-o001_image3_001_asn with 1 products
Rule=candidate_Asn_Image
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '90002'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'niriss'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'clear'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['pupil'], 'value': 'f158m'})
DMSAttrConstraint({


Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmp7waaga44']
Reading rules.
Generating associations.
There where 3 associations and 0 orphaned items found.
Associations found are:
jw93125-o002_spec3_001_asn with 1 products
Rule=candidate_Asn_Spectral
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '93125'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'nirspec'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f290lp'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['grating'], 'value': 'g395m'})
DMSAttrConstraint({'name': 'subarray', 'sources': ['subarray'], 'value': 'full'})
Constraint_Target({'name': 'target', 'sources': ['targetid'], 'value': '1'})
Constraint_Spectral({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'mir_lrs-fixedslit|nrc_grism|nrc_wfss|nrs_autoflat|nrs_autowave|nrs_fixedslit'})
DMSAttrConstraint({'name': 'exp_type', 'sources': ['exp_type


Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpt2ppkikg']
Reading rules.
Generating associations.
There where 10 associations and 31 orphaned items found.
Associations found are:
Association jw86600-o001_image2_001_asn
    Parameters:        Product type: indetermined        Rule:         indetermined        Program:      indetermined        Target:       indetermined        Pool:         indetermined
        DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '86600'})
DMSAttrConstraint({'name': 'is_tso', 'sources': ['tsovisit'], 'value': None})
DMSAttrConstraint({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'fgs_focus'})
SimpleConstraint({'name': 'single_science', 'value': False})
DMSAttrConstraint({'name': 'asn_candidate', 'sources': ['asn_candidate'], 'value': "\\(\\'o001\\'\\,\\ \\'observation\\'\\)"})
	jw86600001001_02101_00005_guider1 with 1 members
	jw86600001001_02101_00003_guider1 with 1 members
	jw866000


Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpj1c645f0']
Reading rules.
Generating associations.
There where 14 associations and 63 orphaned items found.
Associations found are:
jw80600-o006_image3_001_asn with 1 products
Rule=candidate_Asn_Image
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '80600'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'miri'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f2100w'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['pupil', 'grating'], 'value': None})
DMSAttrConstraint({'name': 'subarray', 'sources': ['subarray'], 'value': 'full'})
Constraint_Target({'name': 'target', 'sources': ['targetid'], 'value': '1'})
Constraint_Image({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'mir_image'})
DMSAttrConstraint({'name': 'wfsvisit', 'sources': ['visitype'], 'value': 'prime_targeted_fixed'})
Constraint_Obsnum({'name': 'o

...


Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpj6jbknos']
Reading rules.
Generating associations.
There where 2 associations and 0 orphaned items found.
Associations found are:
jw98005-o001_spec3_001_asn with 1 products
Rule=candidate_Asn_Spectral
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '98005'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'nirspec'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f170lp'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['grating'], 'value': 'g235m'})
DMSAttrConstraint({'name': 'subarray', 'sources': ['subarray'], 'value': 's200a1'})
Constraint_Target({'name': 'target', 'sources': ['targetid'], 'value': '1'})
Constraint_Spectral({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'mir_lrs-fixedslit|nrc_grism|nrc_wfss|nrs_autoflat|nrs_autowave|nrs_fixedslit'})
DMSAttrConstraint({'name': 'exp_type', 'sources': ['exp_ty


There where 5 associations and 0 orphaned items found.
Associations found are:
jw93045-a3001_spec3_001_asn with 1 products
Rule=discover_Asn_Spectral
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '93045'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'nirspec'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f290lp'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['grating'], 'value': 'g395h'})
DMSAttrConstraint({'name': 'subarray', 'sources': ['subarray'], 'value': 's400a1'})
Constraint_Target({'name': 'target', 'sources': ['targetid'], 'value': '1'})
Constraint_Spectral({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'mir_lrs-fixedslit|nrc_grism|nrc_wfss|nrs_autoflat|nrs_autowave|nrs_fixedslit'})
DMSAttrConstraint({'name': 'exp_type', 'sources': ['exp_type'], 'value': '|nrs_autoflat|nrs_autowave|nrs_msaspec'})
DMSAttrConstraint({'name': 'is_msa', 'sources': ['msametfl'], 'value': None})
Const


There where 37 associations and 272 orphaned items found.
Associations found are:
jw82600-a3001_image3_001_asn with 1 products
Rule=discover_Asn_Image
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '82600'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'nircam'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f200w'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['pupil'], 'value': 'maskrnd'})
DMSAttrConstraint({'name': 'subarray', 'sources': ['subarray'], 'value': 'full'})
Constraint_Target({'name': 'target', 'sources': ['targetid'], 'value': '1'})
Constraint_Image({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'nrc_image'})
DMSAttrConstraint({'name': 'wfsvisit', 'sources': ['visitype'], 'value': 'prime_targeted_fixed'})
Constraint_Obsnum({'name': 'obs_num', 'sources': ['obs_num'], 'value': None})
DMSAttrConstraint({'name': 'acq_exp', 'sources': ['exp_type'], 'value': 'mir_tacq|nis_taconfirm|

	jw82600-a3001_t001_nircam_f410m-sub640 with 1 members
Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpyk84vcsq']
Reading rules.
Generating associations.
There where 2 associations and 0 orphaned items found.
Associations found are:
jw90003-o001_image3_001_asn with 1 products
Rule=candidate_Asn_Image
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '90003'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'niriss'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'clear'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['pupil'], 'value': 'f158m'})
DMSAttrConstraint({'name': 'subarray', 'sources': ['subarray'], 'value': 'full'})
Constraint_Target({'name': 'target', 'sources': ['targetid'], 'value': '1'})
Constraint_Image({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'nis_image'})
DMSAttrConstraint({'name': 'wfsvisit', 'sources': ['visitype'], 'value': 'prime_ta


Command-line arguments: ['-v', '-p', '/var/folders/z7/hcykr_n546s_5033t9rj5d200000zj/T/tmpff_a9h7d']
Reading rules.
Generating associations.
There where 4 associations and 0 orphaned items found.
Associations found are:
jw93056-o001_tso3_001_asn with 1 products
Rule=candidate_Asn_TSO
DMSAttrConstraint({'name': 'program', 'sources': ['program'], 'value': '93056'})
DMSAttrConstraint({'name': 'instrument', 'sources': ['instrume'], 'value': 'nirspec'})
Constraint_Target({'name': 'target', 'sources': ['targetid'], 'value': '1'})
DMSAttrConstraint({'name': 'opt_elem', 'sources': ['filter'], 'value': 'f100lp'})
DMSAttrConstraint({'name': 'opt_elem2', 'sources': ['grating'], 'value': 'g140h'})
DMSAttrConstraint({'name': 'subarray', 'sources': ['subarray'], 'value': 'sub2048'})
DMSAttrConstraint({'name': 'exp_type', 'sources': ['exp_type'], 'value': 'nrs_brightobj'})
Constraint_Obsnum({'name': 'obs_num', 'sources': ['obs_num'], 'value': None})
DMSAttrConstraint({'name': 'acq_exp', 'sources': [